In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
import pprint
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as web
from datetime import datetime, timedelta

import requests
from bs4 import BeautifulSoup

In [4]:
yf.Ticker("AAPL").info.get("currentPrice")

219.0998

In [938]:
fcf = apple_stock.get_cash_flow().loc["FreeCashFlow", :].dropna()
years = [fcf.index[i].year for i in range(len(fcf))]
fcf = fcf.set_axis(pd.Index(years)).sort_index().reset_index().rename(columns = {"index":"Year"})
# fcf["Growth"] = fcf["FreeCashFlow"].pct_change()
avg_growth = fcf["FreeCashFlow"].pct_change().mean()
# avg_growth

years_to_project = 5

latest_historical_info = fcf.iloc[-1]
latest_historical_info
future_years = [i for i in range(latest_historical_info["Year"]+1, latest_historical_info["Year"]+years_to_project+1)]

for yr in future_years:
    proj_fcf = (latest_historical_info["FreeCashFlow"] * (1 + avg_growth) ** (yr - latest_historical_info["Year"]))
    fcf.loc[len(fcf.index)] = [yr, proj_fcf]
    # print(fcf.loc[len(fcf.index)])

fcf["Year"] = fcf["Year"].astype(int)
fcf

,Year,FreeCashFlow
0,2020,45234000000.0
1,2021,56118000000.0
2,2022,65149000000.0
3,2023,59475000000.0
4,2024,65709002157.099236
5,2025,72596434879.8936
6,2026,80205788921.742645
7,2027,88612734044.062775
8,2028,97900871499.752533


In [939]:
# FIND ACTUAL VALUES
perpetual_growth_rate = 0.025
discount_rate = 0.08

In [940]:
terminal_value = fcf.iloc[-1]["FreeCashFlow"] * (1 + perpetual_growth_rate) / (discount_rate - perpetual_growth_rate)
terminal_value

1824516241586.297

In [941]:
future_fcf_sum = fcf.tail(years_to_project)["FreeCashFlow"].sum() + terminal_value
future_fcf_sum

2229541073088.8477

In [942]:
balance_sheet = apple_stock.quarterly_balance_sheet
cash_casheq = balance_sheet.loc["Cash Cash Equivalents And Short Term Investments"].iloc[0]
total_debt = balance_sheet.loc["Total Debt"].iloc[0]
equity_value = future_fcf_sum + cash_casheq - total_debt
shares_outstanding = balance_sheet.loc["Share Issued"].iloc[0]

dcf_pps = (equity_value / shares_outstanding)
dcf_pps

299.9654342915172

In [943]:
yf.Ticker("^TNX").info.get("previousClose") / 100

0.04238

In [955]:
data = web.DataReader('DGS10', 'fred', datetime.datetime(2023, 1, 1), datetime.datetime.now())
data.iloc[-1]["DGS10"] / 100

0.0431

In [957]:
yf.Ticker("AAPL").financials

,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Tax Effect Of Unusual Items,0.0,0.0,0.0,0.0
Tax Rate For Calcs,0.147,0.162,0.133,0.144
Normalized EBITDA,129188000000.0,133138000000.0,123136000000.0,81020000000.0
Net Income From Continuing Operation Net Minority Interest,96995000000.0,99803000000.0,94680000000.0,57411000000.0
Reconciled Depreciation,11519000000.0,11104000000.0,11284000000.0,11056000000.0
Reconciled Cost Of Revenue,214137000000.0,223546000000.0,212981000000.0,169559000000.0
EBITDA,129188000000.0,133138000000.0,123136000000.0,81020000000.0
EBIT,117669000000.0,122034000000.0,111852000000.0,69964000000.0
Net Interest Income,-183000000.0,-106000000.0,198000000.0,890000000.0
Interest Expense,3933000000.0,2931000000.0,2645000000.0,2873000000.0


In [10]:
# Sustained growth rate
risk_free_rate = yf.Ticker("^TNX").info.get("previousClose") / 100
sp500 = yf.download("^GSPC", 
                    start = "1996-11-22",#(datetime.now() - timedelta(days = 10000)).strftime("%Y-%m-%d"), 
                    end = datetime.now().strftime("%Y-%m-%d"))
market_return = ((1 + sp500["Adj Close"].pct_change().dropna().mean()) ** 252) - 1


def find_SGR(ticker):
    try:
        stock = yf.Ticker(ticker)
        ROE = stock.info.get("returnOnEquity")
        dividends_paid = abs(stock.cashflow.loc["Cash Dividends Paid"].head(1).values[0])
        net_income = stock.incomestmt.loc["Net Income"].head(1).values[0]
        retention_rate = 1 - (dividends_paid / net_income)
        
        return ROE * retention_rate
    
    except:
        return 100 # arbitrary maximum

# Long-term growth rate (nominal interest rate)
def find_LTGR():
    data = web.DataReader('DGS10', 'fred', datetime(2023, 1, 1), datetime.now())
    return data.iloc[-1]["DGS10"] / 100

def find_WACC(ticker):
    stock = yf.Ticker(ticker)
    beta = stock.info.get("beta")
    cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)

    interest_expense = stock.financials.loc["Interest Expense"].head(1).values[0]
    total_debt = stock.balancesheet.loc["Total Debt"].head(1).values[0]
    cost_of_debt = interest_expense / total_debt

    market_cap = stock.info.get("marketCap")
    equity_weight = market_cap / (market_cap + total_debt)
    debt_weight = 1 - equity_weight
    tax_rate = stock.financials.loc["Tax Rate For Calcs"].head(1).values[0]

    wacc = (cost_of_equity * equity_weight) + (cost_of_debt * debt_weight * (1 - tax_rate))
    """print("Risk-Free Rate:", risk_free_rate)
    print("Market Return:", market_return)
    print("Equity Market Risk Premium:", (market_return - risk_free_rate))
    print("Beta:", beta)
    print("Cost of Equity:", cost_of_equity)
    print("Equity Weight:", equity_weight)
    print("Cost of Debt:", cost_of_debt)
    print("Debt Weight:", debt_weight)
    print("Tax Rate:", tax_rate)"""
    return wacc


[*********************100%%**********************]  1 of 1 completed


In [12]:
def DCF(ticker, years_to_project = 10):
    stock = yf.Ticker(ticker)
    fcf = stock.get_cash_flow().loc["FreeCashFlow", :].dropna()
    past_years = [fcf.index[i].year for i in range(len(fcf))]
    fcf = fcf.set_axis(pd.Index(past_years)).sort_index().reset_index().rename(columns = {"index":"Year"})
    
    avg_growth = fcf["FreeCashFlow"].pct_change().mean()

    latest_historical_info = fcf.iloc[-1]
    future_years = [i for i in range(latest_historical_info["Year"] + 1, latest_historical_info["Year"] + years_to_project + 1)]

    for yr in future_years:
        proj_fcf = (latest_historical_info["FreeCashFlow"] * (1 + avg_growth) ** (yr - latest_historical_info["Year"]))
        fcf.loc[len(fcf.index)] = [yr, proj_fcf]

    perpetual_growth_rate = min(find_SGR(ticker), find_LTGR())
    discount_rate = find_WACC(ticker)

    terminal_value = fcf.iloc[-1]["FreeCashFlow"] * (1 + perpetual_growth_rate) / (discount_rate - perpetual_growth_rate)
    future_fcf_sum = fcf.tail(years_to_project)["FreeCashFlow"].sum() + terminal_value

    balance_sheet = apple_stock.quarterly_balance_sheet
    cash_casheq = balance_sheet.loc["Cash Cash Equivalents And Short Term Investments"].iloc[0]
    total_debt = balance_sheet.loc["Total Debt"].iloc[0]
    equity_value = future_fcf_sum + cash_casheq - total_debt
    shares_outstanding = balance_sheet.loc["Share Issued"].iloc[0]
    
    dcf_pps = (equity_value / shares_outstanding)
    return round(dcf_pps, 2)

In [14]:
def compare(ticker, years_to_project = 10):
    comp_stock = yf.Ticker(ticker)
    curr_price = comp_stock.info.get("previousClose")
    DCF_price = DCF(ticker, years_to_project)
    comp_name = comp_stock.info.get("shortName")
    return comp_name + " is currently at $" + str(curr_price) + ", and its DCF projection is $" + str(DCF_price)

In [967]:
# print(DCF("AAPL"))
compare("AAPL")

'Apple Inc. is currently at $214.24, and its DCF projection is $954.55'

In [11]:
resp = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = {}
for row in table.findAll('tr')[1:]:
    # print(row)
    # print("\n")
    ticker = row.findAll('td')[0].text.replace("\n", "")
    name = row.findAll('td')[1].text.replace("\n", "")
    # print(ticker + ":", name)
    tickers[ticker] = name

    # tickers.append(ticker)
tickers_lst = []
for key, value in tickers.items():
    entry = value + " (" + key + ")"

MMM: 3M
AOS: A. O. Smith
ABT: Abbott
ABBV: AbbVie
ACN: Accenture
ADBE: Adobe Inc.
AMD: Advanced Micro Devices
AES: AES Corporation
AFL: Aflac
A: Agilent Technologies
APD: Air Products and Chemicals
ABNB: Airbnb
AKAM: Akamai
ALB: Albemarle Corporation
ARE: Alexandria Real Estate Equities
ALGN: Align Technology
ALLE: Allegion
LNT: Alliant Energy
ALL: Allstate
GOOGL: Alphabet Inc. (Class A)
GOOG: Alphabet Inc. (Class C)
MO: Altria
AMZN: Amazon
AMCR: Amcor
AEE: Ameren
AAL: American Airlines Group
AEP: American Electric Power
AXP: American Express
AIG: American International Group
AMT: American Tower
AWK: American Water Works
AMP: Ameriprise Financial
AME: Ametek
AMGN: Amgen
APH: Amphenol
ADI: Analog Devices
ANSS: Ansys
AON: Aon
APA: APA Corporation
AAPL: Apple Inc.
AMAT: Applied Materials
APTV: Aptiv
ACGL: Arch Capital Group
ADM: Archer-Daniels-Midland
ANET: Arista Networks
AJG: Arthur J. Gallagher & Co.
AIZ: Assurant
T: AT&T
ATO: Atmos Energy
ADSK: Autodesk
ADP: Automatic Data Processing


[]

In [972]:
for ticker in tickers:
    try:
        print(ticker)
        print(compare(ticker))
    except Exception as e:
        print(e)
        continue

MMM
3M Company is currently at $101.54, and its DCF projection is $5.29
AOS
A.O. Smith Corporation is currently at $83.67, and its DCF projection is $9.67
ABT
Abbott Laboratories is currently at $103.45, and its DCF projection is $32.79
ABBV
AbbVie Inc. is currently at $166.56, and its DCF projection is $58.21
ACN
Accenture plc is currently at $282.32, and its DCF projection is $52.73
ADBE
Adobe Inc. is currently at $458.74, and its DCF projection is $56.88
AMD
Advanced Micro Devices, Inc. is currently at $159.9, and its DCF projection is $871.25
AES
The AES Corporation is currently at $19.6, and its DCF projection is $-5583826.88
AFL
AFLAC Incorporated is currently at $87.31, and its DCF projection is $3.03
A
Agilent Technologies, Inc. is currently at $130.57, and its DCF projection is $58.7
APD
Air Products and Chemicals, Inc is currently at $285.31, and its DCF projection is $-10.48
ABNB
Airbnb, Inc. is currently at $146.1, and its DCF projection is $-0.1
AKAM
Akamai Technologies, I

In [974]:
yf.Ticker("WFC").get_cash_flow()

,2023-12-31,2022-12-31,2021-12-31,2020-12-31,2019-12-31
FreeCashFlow,40358000000.0,27048000000.0,-11525000000.0,2051000000.0,NaN
RepurchaseOfCapitalStock,-13576000000.0,-6033000000.0,-21139000000.0,-7017000000.0,NaN
RepaymentOfDebt,-22886000000.0,-19587000000.0,-47134000000.0,-65347000000.0,NaN
IssuanceOfDebt,49071000000.0,53737000000.0,1275000000.0,38136000000.0,NaN
IssuanceOfCapitalStock,1722000000.0,0.0,5756000000.0,3116000000.0,NaN
InterestPaidSupplementalData,30431000000.0,8289000000.0,4384000000.0,8414000000.0,NaN
IncomeTaxPaidSupplementalData,NaN,3376000000.0,3166000000.0,1175000000.0,7493000000.0
EndCashPosition,236052000000.0,159157000000.0,234230000000.0,264612000000.0,NaN
BeginningCashPosition,159157000000.0,234230000000.0,264612000000.0,141250000000.0,NaN
ChangesInCash,76895000000.0,-75073000000.0,-30382000000.0,123362000000.0,NaN


In [997]:
url = f"https://www.macrotrends.net/stocks/charts/WFC/wells-fargo/free-cash-flow"

# Send a GET request to fetch the raw HTML content
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table containing the cash flow data
tables = soup.find_all('table')

In [50]:
AV_apikey = "X5KOQR5HRJ1NMVD9"
ticker = "AAPL"

In [52]:
url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={ticker}&apikey={AV_apikey}'

In [54]:
link = 'https://www.alphavantage.co/query?function=CASH_FLOW&symbol=IBM&apikey=demo'
r = requests.get(url)
data = r.json()
pprint.pprint(data)

{'annualReports': [{'capitalExpenditures': '10959000000',
                    'cashflowFromFinancing': '-108488000000',
                    'cashflowFromInvestment': '3705000000',
                    'changeInCashAndCashEquivalents': '5760000000',
                    'changeInExchangeRate': 'None',
                    'changeInInventory': '1618000000',
                    'changeInOperatingAssets': '7719000000',
                    'changeInOperatingLiabilities': '1142000000',
                    'changeInReceivables': '417000000',
                    'depreciationDepletionAndAmortization': '11519000000',
                    'dividendPayout': '15025000000',
                    'dividendPayoutCommonStock': '15025000000',
                    'dividendPayoutPreferredStock': 'None',
                    'fiscalDateEnding': '2023-09-30',
                    'netIncome': '96995000000',
                    'operatingCashflow': '110543000000',
                    'paymentsForOperatingActivities

In [60]:
data["annualReports"][0]
# test = data["annualReports"][0]
# test["fiscalDateEnding"][:4]

{'fiscalDateEnding': '2023-09-30',
 'reportedCurrency': 'USD',
 'operatingCashflow': '110543000000',
 'paymentsForOperatingActivities': '5703000000',
 'proceedsFromOperatingActivities': 'None',
 'changeInOperatingLiabilities': '1142000000',
 'changeInOperatingAssets': '7719000000',
 'depreciationDepletionAndAmortization': '11519000000',
 'capitalExpenditures': '10959000000',
 'changeInReceivables': '417000000',
 'changeInInventory': '1618000000',
 'profitLoss': '96995000000',
 'cashflowFromInvestment': '3705000000',
 'cashflowFromFinancing': '-108488000000',
 'proceedsFromRepaymentsOfShortTermDebt': '-7956000000',
 'paymentsForRepurchaseOfCommonStock': '77550000000',
 'paymentsForRepurchaseOfEquity': '77550000000',
 'paymentsForRepurchaseOfPreferredStock': 'None',
 'dividendPayout': '15025000000',
 'dividendPayoutCommonStock': '15025000000',
 'dividendPayoutPreferredStock': 'None',
 'proceedsFromIssuanceOfCommonStock': 'None',
 'proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet

In [62]:
df = pd.DataFrame({"Year": [],
                   "OperatingCashFlow": [],
                   "CapitalExpenditure": [],
                   "FreeCashFlow": []})

for dict in data["annualReports"]:
    year = dict["fiscalDateEnding"][:4]
    ocf = int(dict["operatingCashflow"])
    capex = int(dict["capitalExpenditures"])
    fcf = ocf - capex
    entry = [year, ocf, capex, fcf]
    df.loc[len(df.index)] = entry

df

,Year,OperatingCashFlow,CapitalExpenditure,FreeCashFlow
0,2023,110543000000,10959000000,99584000000
1,2022,122151000000,10708000000,111443000000
2,2021,104038000000,11085000000,92953000000
3,2020,80674000000,7309000000,73365000000
4,2019,69391000000,10495000000,58896000000
5,2018,77434000000,13313000000,64121000000
6,2017,64225000000,12795000000,51430000000
7,2016,66231000000,13548000000,52683000000
8,2015,81266000000,11488000000,69778000000
9,2014,109133000000,9571000000,99562000000
